In [1]:
%load_ext autoreload
%autoreload 2
import sys
sys.path.append("..")
import torch
from dnn_training.dataset import *
import numpy as np
from dnn_training.trainer import *
import awkward as ak

In [6]:
device="cuda:1"
model = makeModel()
loss = BinaryLoss()
trainer = Trainer(model.to(device), loss=loss, device=device, log_output="/grid_mnt/data_cms_upgrade/cuisset/supercls/dnn_training/test")
datasets_binary = makeDatasetsTrainVal_fromCache("/grid_mnt/data_cms_upgrade/cuisset/supercls/alessandro_electrons/supercls-v15-sampleDump/superclsDumper_*.root", device_valDataset=device)
datasets_continuous = makeDatasetsTrainVal_fromCache("/grid_mnt/data_cms_upgrade/cuisset/supercls/alessandro_electrons/supercls-v15-sampleDump/superclsDumper_*.root", device_valDataset=device, trainingLossType="continuousAssociationScore")

In [11]:
datasets_binary["valDataset"]["seedEnergy_perEvent"]

tensor([226.0455, 377.6656, 215.2264,  ..., 318.1190,  64.6889, 235.1924])

In [4]:
weighted_val_indices = torch.tensor(list(WeightedRandomSampler([1, 2, 3, 4, 5], 2, replacement=True)))
weighted_val_indices

tensor([4, 3])

In [5]:
datasets_binary["valDataset"]["features"]

tensor([[ 2.2780e-02,  1.4527e+00,  8.7256e+00,  ...,  3.9218e-01,
          1.5403e+01,  9.7217e-01],
        [-5.7643e-02,  4.5052e-01,  4.4632e+00,  ...,  1.4441e-01,
          1.9119e+01,  9.3179e-01],
        [ 1.1926e-01,  6.6634e-02,  5.9421e+00,  ...,  7.5706e-02,
          1.6931e+01,  9.7667e-01],
        ...,
        [ 1.2865e-02, -3.3939e+00,  2.6543e+00,  ...,  2.5400e-01,
          1.6913e+01,  9.1402e-01],
        [ 8.7223e-02,  1.2628e+00,  2.4664e+00,  ...,  1.9005e-01,
          1.5394e+01,  9.8801e-01],
        [ 1.1684e-01,  1.3013e+00,  1.9700e+00,  ...,  2.6743e-01,
          1.9028e+01,  9.3996e-01]])

In [6]:
datasets_binary["valDataset"]["features"][weighted_val_indices]

tensor([[-5.9391e-02,  1.5638e+00,  3.5737e+00,  1.6228e+00,  1.3577e+00,
          1.6822e+00, -1.7609e+00,  2.2605e+02,  8.5876e+01,  2.5343e-01,
          7.7267e-01,  7.7267e-01,  5.6504e-01,  9.8422e-02,  2.5585e-01,
          1.3927e+01,  9.7030e-01],
        [ 8.1895e-02,  2.3290e-01,  4.5126e+00,  1.7641e+00,  1.5023e+00,
          1.6822e+00, -1.7609e+00,  2.2605e+02,  7.5256e+01,  1.1014e-01,
          7.2830e-01,  7.2830e-01,  2.9245e-01,  3.5116e-02,  1.1165e-01,
          2.2821e+01,  9.9193e-01]])

In [7]:
def histedges_equalN(x, nbin):
    npt = len(x)
    return np.interp(np.linspace(0, npt, nbin + 1),
                     np.arange(npt),
                     np.sort(x))
histedges_equalN(datasets_binary["valDataset"]["seedEnergy_perEvent"], 5)

array([ 16.56748009, 110.09165649, 185.76254578, 260.24767456,
       366.69959106, 764.33905029])

In [13]:
val_dataset_gpu = {key : tensor.to(device) for key, tensor in datasets_binary["valDataset"].items()}

In [14]:
val_dataset_gpu

{'features': tensor([[ 2.2780e-02,  1.4527e+00,  8.7256e+00,  ...,  3.9218e-01,
           1.5403e+01,  9.7217e-01],
         [-5.7643e-02,  4.5052e-01,  4.4632e+00,  ...,  1.4441e-01,
           1.9119e+01,  9.3179e-01],
         [ 1.1926e-01,  6.6634e-02,  5.9421e+00,  ...,  7.5706e-02,
           1.6931e+01,  9.7667e-01],
         ...,
         [ 1.2865e-02, -3.3939e+00,  2.6543e+00,  ...,  2.5400e-01,
           1.6913e+01,  9.1402e-01],
         [ 8.7223e-02,  1.2628e+00,  2.4664e+00,  ...,  1.9005e-01,
           1.5394e+01,  9.8801e-01],
         [ 1.1684e-01,  1.3013e+00,  1.9700e+00,  ...,  2.6743e-01,
           1.9028e+01,  9.3996e-01]], device='cuda:1'),
 'genmatching': tensor([0., 0., 0.,  ..., 0., 0., 0.], device='cuda:1'),
 'assocScore_training': tensor([1.0000, 1.0000, 0.9999,  ..., 1.0000, 1.0000, 1.0000], device='cuda:1'),
 'seedTracksterBestAssociationScore': tensor([0.0002, 0.0002, 0.0002,  ..., 0.0004, 0.0004, 0.0004], device='cuda:1'),
 'candidateTracksterAssociat

In [10]:
import optuna

In [9]:

trainer.full_train(datasets_binary["trainDataset"], datasets_binary["valDataset"], weightSamples=True, nepochs=15, batch_size=512)

  0%|          | 0/15 [00:00<?, ?it/s]

train_loss = 0.07459397972232915
val_loss = 0.10559932887554169
train_loss = 0.07292086761233091
val_loss = 0.09296827018260956
train_loss = 0.07167070469212586
val_loss = 0.09713432192802429
train_loss = 0.07051104404166235
val_loss = 0.08664406090974808
train_loss = 0.06964493629581292
val_loss = 0.10447023808956146
train_loss = 0.06853648419796278
val_loss = 0.10663112998008728
train_loss = 0.06803990680916154
val_loss = 0.09349510818719864
train_loss = 0.0671719534018716
val_loss = 0.09372750669717789
train_loss = 0.06669555090037405
val_loss = 0.09271429479122162
train_loss = 0.06622156195675916
val_loss = 0.09756290167570114
train_loss = 0.06587322131462552
val_loss = 0.09583238512277603
train_loss = 0.06533828528707931
val_loss = 0.09308748692274094
train_loss = 0.06507281476881709
val_loss = 0.08940528333187103
train_loss = 0.06500072271600447
val_loss = 0.0952853262424469
train_loss = 0.06436894056211796
val_loss = 0.0894392877817154
Exported graph: graph(%input : Float(*, 17,

In [31]:
trainer_continuous = Trainer(model.to(device), loss=ContinuousAssociationScoreLoss(), device=device, log_output="/grid_mnt/data_cms_upgrade/cuisset/supercls/dnn_training/test")
trainer_continuous.full_train(datasets_continuous["trainDataset"], datasets_continuous["valDataset"])

  0%|          | 0/10 [00:00<?, ?it/s]

train_loss = 0.012860737525821852
val_loss = 6.490610464476477e-09
train_loss = 0.005304191021536462
val_loss = 5.4315321155173905e-09
train_loss = 0.004812510449237986
val_loss = 5.2447152801642364e-09
train_loss = 0.004631943809679507
val_loss = 5.098495566149538e-09
train_loss = 0.004525918076884276
val_loss = 5.057942733599121e-09


Exception in thread Thread-36 (_pin_memory_loop):
Traceback (most recent call last):
  File "/grid_mnt/data_cms_upgrade/cuisset/conda/envs/ticlRegression-gpu/lib/python3.11/threading.py", line 1045, in _bootstrap_inner
    self.run()
  File "/grid_mnt/data_cms_upgrade/cuisset/conda/envs/ticlRegression-gpu/lib/python3.11/site-packages/ipykernel/ipkernel.py", line 761, in run_closure
    _threading_Thread_run(self)
  File "/grid_mnt/data_cms_upgrade/cuisset/conda/envs/ticlRegression-gpu/lib/python3.11/threading.py", line 982, in run
    self._target(*self._args, **self._kwargs)
  File "/grid_mnt/data_cms_upgrade/cuisset/conda/envs/ticlRegression-gpu/lib/python3.11/site-packages/torch/utils/data/_utils/pin_memory.py", line 53, in _pin_memory_loop
    do_one_step()
  File "/grid_mnt/data_cms_upgrade/cuisset/conda/envs/ticlRegression-gpu/lib/python3.11/site-packages/torch/utils/data/_utils/pin_memory.py", line 30, in do_one_step
    r = in_queue.get(timeout=MP_STATUS_CHECK_INTERVAL)
       

KeyboardInterrupt: 

In [29]:
datasets_continuous["valDataset"]["assocScore_training"]

tensor([1., 1., 1.,  ..., 1., 1., 1.])

In [23]:
gen = datasets_continuous["trainDataset"][:]["assocScore_training"][0] <= 0.2
#sampler = WeightedRandomSampler(torch.where(gen==1, torch.sum(gen==0)/torch.sum(gen==1), 1.), gen.shape[0], replacement=True)
torch.sum(gen) / len(gen)

tensor(0.0072)

In [13]:
train_dataloader = DataLoader(datasets_binary["trainDataset"], batch_size=5,  sampler=sampler)
next(iter(train_dataloader))["genmatching"][0]

tensor([0., 1., 1., 0., 0.])

In [11]:
datasets_continuous["trainDataset"][:]["assocScore_training"][0]

tensor([1., 1., 1.,  ..., 1., 1., 1.])

In [9]:
WeightedRandomSampler()

torch.utils.data.sampler.WeightedRandomSampler

## Loading already trained model

In [6]:
model = makeModel()
loss = BinaryLoss()
trainer = Trainer(model, loss=loss, device="cpu", log_output="/grid_mnt/data_cms_upgrade/cuisset/supercls/dnn_training/test")
trainer.reloadModel("/grid_mnt/data_cms_upgrade/cuisset/supercls/dnn_training/full-v2/Apr05_14-48-03/model_epoch376.pth")

In [7]:
trainer.model.eval()
trainer.saveModel("epoch376", format="onnx")

Exported graph: graph(%input : Float(*, 17, strides=[17, 1], requires_grad=0, device=cpu),
      %0.scaler_scale : Float(17, strides=[1], requires_grad=0, device=cpu),
      %0.scaler_min : Float(17, strides=[1], requires_grad=0, device=cpu),
      %1.weight : Float(100, 17, strides=[17, 1], requires_grad=1, device=cpu),
      %1.bias : Float(100, strides=[1], requires_grad=1, device=cpu),
      %4.weight : Float(200, 100, strides=[100, 1], requires_grad=1, device=cpu),
      %4.bias : Float(200, strides=[1], requires_grad=1, device=cpu),
      %7.weight : Float(100, 200, strides=[200, 1], requires_grad=1, device=cpu),
      %7.bias : Float(100, strides=[1], requires_grad=1, device=cpu),
      %10.weight : Float(1, 100, strides=[100, 1], requires_grad=1, device=cpu),
      %10.bias : Float(1, strides=[1], requires_grad=1, device=cpu)):
  %/0/Mul_output_0 : Float(*, 17, strides=[17, 1], requires_grad=0, device=cpu) = onnx::Mul[onnx_name="/0/Mul"](%input, %0.scaler_scale), scope: torch.n

In [2]:
device = "cuda:1"

In [3]:
model, loss = makeModelLoss()
trainer = Trainer(model.to(device), loss, device=device, log_output="/grid_mnt/data_cms_upgrade/cuisset/supercls/dnn_training")

In [4]:
train_dataset, val_dataset = makeDatasetsTrainVal("/grid_mnt/data_cms_upgrade/cuisset/supercls/alessandro_electrons/supercls-v13/superclsDumper_1.root", device="cpu")

In [10]:
trainer.full_train(train_dataset, val_dataset, nepochs=5)

  0%|          | 0/5 [00:00<?, ?it/s]

train_loss = 0.30187917076546
val_loss = 0.010887423530220985
train_loss = 0.30434493648499483
val_loss = 0.011025166371837258
train_loss = 0.28661539388122037
val_loss = 0.010727119981311262
train_loss = 0.3006898802414071
val_loss = 0.013658439042046666
train_loss = 0.2873406008075108
val_loss = 0.0105202526319772


In [6]:
trainer.scheduler.get_last_lr()

[0.001]

In [11]:
import torch.onnx

In [27]:
train_dataset[0:2]["features"][0].dtype

torch.float32

In [28]:
torch.ones((1, 17)).dtype

torch.float32

In [22]:
model.to("cpu")

torch.onnx.export(
    model,
    train_dataset[0:2]["features"][0].to("cpu"),
    "model.onnx",
    export_params=True,
    verbose=True,
    input_names=["input"],
    output_names=["output"],
    opset_version=10,
    dynamic_axes={"input" : {0:"batch_size"}, "output": {0:"batch_size"}}
)

Exported graph: graph(%input : Float(*, 17, strides=[17, 1], requires_grad=0, device=cpu),
      %0.weight : Float(100, 17, strides=[17, 1], requires_grad=1, device=cpu),
      %0.bias : Float(100, strides=[1], requires_grad=1, device=cpu),
      %2.weight : Float(200, 100, strides=[100, 1], requires_grad=1, device=cpu),
      %2.bias : Float(200, strides=[1], requires_grad=1, device=cpu),
      %4.weight : Float(100, 200, strides=[200, 1], requires_grad=1, device=cpu),
      %4.bias : Float(100, strides=[1], requires_grad=1, device=cpu),
      %6.weight : Float(1, 100, strides=[100, 1], requires_grad=1, device=cpu),
      %6.bias : Float(1, strides=[1], requires_grad=1, device=cpu)):
  %/0/Gemm_output_0 : Float(*, 100, strides=[100, 1], requires_grad=1, device=cpu) = onnx::Gemm[alpha=1., beta=1., transB=1, onnx_name="/0/Gemm"](%input, %0.weight, %0.bias), scope: torch.nn.modules.container.Sequential::/torch.nn.modules.linear.Linear::0 # /grid_mnt/data_cms_upgrade/cuisset/conda/envs/ti